# Pipeline

In [2]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

role = sagemaker.get_execution_role()
bucket = "pochingto-testing"
pipeline_session = PipelineSession(default_bucket=bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
sagemaker.__version__

'2.196.0'

In [4]:
!pip3 install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.0/917.0 kB 5.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.197.0-py2.py3-none-any.whl size=1223345 sha256=cf0100ec219c94eac16314c1cfa4bdd6830836595c08fa9dddae415858eb0a02
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/cb/f7/c8/c7d6a8a2b801fda3c6fada9982293887f45390a9b8e0cb4eb3
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.196.0
    Uninstalling sagemaker-2.196.0:
      Successfully uninstalled sagemaker-2.196.0


In [5]:
import sagemaker
import importlib
importlib.reload(sagemaker)
sagemaker.__version__

'2.197.0'

In [6]:
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name
bucket = "pochingto-testing"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [7]:
config = {
    "session": pipeline_session,
    "instance_type": "ml.m5.xlarge",
    "image": None,
    "framework_version": "1.12",
    "py_version": "py38",
}

In [8]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

## Data preprocessing

In [9]:
from sagemaker.workflow.parameters import ParameterString

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"s3://{bucket}/all",
)

In [10]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

In [11]:
dataset_location

ParameterString(name='dataset_location', parameter_type=<ParameterTypeEnum.STRING: 'String'>, default_value='s3://pochingto-testing/all')

In [12]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.sklearn.processing import SKLearnProcessor

# Define the script processor
sklearn_processor = SKLearnProcessor(
    base_job_name="preprocess-data",
    framework_version="1.2-1",
    role=role,  # Replace with your SageMaker role ARN
    instance_count=1,
    instance_type=config["instance_type"],
    sagemaker_session=config["session"]
)

preprocessing_step = ProcessingStep(
    name="preprocess-data",
    step_args=sklearn_processor.run(
        code='split_data.py',
        inputs=[
            ProcessingInput(
                source=dataset_location,
                destination='/opt/ml/processing/input'
            )
        ],
        outputs=[
            ProcessingOutput(
                output_name="train",
                source='/opt/ml/processing/output/train',
                destination=f's3://{bucket}/output/train'
            ),
            ProcessingOutput(
                output_name="test",
                source='/opt/ml/processing/output/test',
                destination=f's3://{bucket}/output/test'
            )
        ]
    ),
    cache_config=cache_config
)

/opt/conda/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [13]:
type(preprocessing_step)

sagemaker.workflow.steps.ProcessingStep

In [14]:
# dogbreed_pipeline = Pipeline(
#     name="dogbreeds-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         preprocessing_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )

# dogbreed_pipeline.upsert(role_arn=role)

## Training

In [15]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    base_job_name="dogbreeds-training",
    entry_point=f"train.py",

    hyperparameters={
        "epochs": 5,
        "batch_size": 32,
    },
    
    metric_definitions=[
        {"Name": "loss", "Regex": "Loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "Validation Accuracy: ([0-9\\.]+)"},
    ],
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type="ml.g4dn.xlarge",
    instance_count=1,
    disable_profiler=True,
    sagemaker_session=config["session"],
    role=role
)

In [16]:
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput

train_model_step = TrainingStep(
    name="train-model",
    step_args=estimator.fit(
        inputs={
            "train": TrainingInput(
                s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri
            )
        }
    ),
    cache_config=cache_config,
)

## Evaluation

In [17]:
!ls

__MACOSX		   dogBreeds.flow     lambda.py		 test_images
__pycache__		   dogBreeds.zip      model.pt		 train.py
all			   evaluation.py      model.tar.gz	 untitled.flow
all.tar.gz		   experiments.ipynb  non_image_folders
best_model_checkpoint.pth  inference.py       split_data.py


In [18]:
# if 'autoreload' not in get_ipython().extension_manager.loaded:
#     %load_ext autoreload
import importlib

import tempfile
import shutil

from pathlib import Path
import evaluation

importlib.reload(evaluation)

num_epochs = 5
batch_size = 32

directory = tempfile.mkdtemp()
output_directory = Path(directory) / "output"
output_directory.mkdir(parents=True, exist_ok=True)

evaluation.main("./", "all/", output_directory)

Loading model...
Context:  Evaluation
extracted tar ...


KeyboardInterrupt: 

In [19]:
print(directory)
!cat /tmp/tmpf8kvn5ak/output/evaluation.json

/tmp/tmpn_fnergt
cat: /tmp/tmpf8kvn5ak/output/evaluation.json: No such file or directory


In [20]:
shutil.rmtree(directory)

In [21]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

In [22]:
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts
model_assets

In [23]:
config

{'session': <sagemaker.workflow.pipeline_context.PipelineSession at 0x7f6d13802850>,
 'instance_type': 'ml.m5.xlarge',
 'image': None,
 'framework_version': '1.12',
 'py_version': 'py38'}

In [24]:
from sagemaker.pytorch.processing import PyTorchProcessor

pytorch_processor = PyTorchProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)


In [25]:
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=pytorch_processor.run(
        inputs=[
            ProcessingInput(
                source=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

/opt/conda/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Registering Model

In [26]:
config

{'session': <sagemaker.workflow.pipeline_context.PipelineSession at 0x7f6d13802850>,
 'instance_type': 'ml.m5.xlarge',
 'image': None,
 'framework_version': '1.12',
 'py_version': 'py38'}

In [27]:
MODEL_PACKAGE_GROUP = "dogBreeds"

In [28]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=model_assets,
    entry_point="inference.py",
    image_uri=config["image"],
    py_version=config["py_version"],
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

In [29]:
from sagemaker.model_metrics import ModelMetrics, MetricsSource
from sagemaker.workflow.functions import Join

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs[
                    "evaluation"
                ].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    )
)

In [30]:
from sagemaker.workflow.model_step import ModelStep

register_model_step = ModelStep(
    name="register-model",
    step_args=pytorch_model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        approval_status="PendingManualApproval",
        model_metrics=model_metrics,
        content_types=["application/x-image"],
        response_types=["application/json"],
        inference_instances=["ml.m5.xlarge"],
        transform_instances=["ml.g4dn.xlarge"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version=config["framework_version"],
    ),
)

In [31]:
from sagemaker.workflow.parameters import ParameterFloat

accuracy_threshold = ParameterFloat(name="accuracy_threshold", default_value=0.50)

In [32]:
from sagemaker.workflow.fail_step import FailStep

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's accuracy was lower than",
            accuracy_threshold,
        ],
    ),
)

In [33]:
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo

condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluate_model_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value",
    ),
    right=accuracy_threshold,
)

In [34]:
from sagemaker.workflow.condition_step import ConditionStep

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [35]:
training_pipeline = Pipeline(
    name="dogBreeds-training-pipeline",
    parameters=[dataset_location, accuracy_threshold],
    steps=[
        preprocessing_step,
        train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

training_pipeline.upsert(role_arn=role)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://pochingto-testing/dogBreeds-training-pipeline/code/648e41a8b10d09dc29265caf05c3e0eb/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://pochingto-testing/dogBreeds-training-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://pochingto-testing/dogBreeds-training-pipeline/code/648e41a8b10d09dc29265caf05c3e0eb/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://pochingto-testing/dogBreeds-training-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:681340771742:pipeline/dogBreeds-training-pipeline',
 'ResponseMetadata': {'RequestId': '9deebc26-b67a-4db6-9112-e765760844bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9deebc26-b67a-4db6-9112-e765760844bb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Sat, 11 Nov 2023 03:19:10 GMT'},
  'RetryAttempts': 0}}

# Setup lambda

In [37]:
# setup role for lambda to deploy endpoint
import json

lambda_role_name = "lambda-deployment-role"
lambda_role_arn = None

try:
    response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(
            {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": ["lambda.amazonaws.com", "events.amazonaws.com"]
                        },
                        "Action": "sts:AssumeRole",
                    }
                ],
            }
        ),
        Description="Lambda Endpoint Deployment",
    )

    lambda_role_arn = response["Role"]["Arn"]

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
        RoleName=lambda_role_name,
    )

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
        RoleName=lambda_role_name,
    )

    print(f'Role "{lambda_role_name}" created with ARN "{lambda_role_arn}".')
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=lambda_role_name)
    lambda_role_arn = response["Role"]["Arn"]
    print(f'Role "{lambda_role_name}" already exists with ARN "{lambda_role_arn}".')

ClientError: An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:sts::681340771742:assumed-role/AmazonSageMaker-ExecutionRole-20230916T122655/SageMaker is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::681340771742:role/lambda-deployment-role because no identity-based policy allows the iam:CreateRole action

In [40]:
bucket

'pochingto-testing'

In [41]:
from sagemaker.predictor import Predictor

ENDPOINT = "dogbreeds-endpoint"
DATA_CAPTURE_DESTINATION = f"{bucket}/monitoring/data-capture"

In [43]:
DATA_CAPTURE_DESTINATION

'pochingto-testing/monitoring/data-capture'

In [44]:
role

'arn:aws:iam::681340771742:role/service-role/AmazonSageMaker-ExecutionRole-20230916T122655'

In [42]:
from sagemaker.lambda_helper import Lambda

lambda_role_arn = "arn:aws:iam::681340771742:role/lambda-deployment-role"
deploy_lambda_fn = Lambda(
    function_name="deploy_fn",
    execution_role_arn=lambda_role_arn,
    script="lambda.py",
    handler="lambda.lambda_handler",
    timeout=600,
    session=sagemaker_session,
    runtime="python3.11",
    environment={
        "Variables": {
            "ENDPOINT": ENDPOINT,
            "DATA_CAPTURE_DESTINATION": DATA_CAPTURE_DESTINATION,
            "ROLE": role,
        }
    },
)

lambda_response = deploy_lambda_fn.upsert()
lambda_response

ValueError: {'Message': 'User: arn:aws:sts::681340771742:assumed-role/AmazonSageMaker-ExecutionRole-20230916T122655/SageMaker is not authorized to perform: lambda:CreateFunction on resource: arn:aws:lambda:us-east-1:681340771742:function:deploy_fn because no identity-based policy allows the lambda:CreateFunction action', 'Code': 'AccessDeniedException'}

## Setup Eventbridge

In [46]:
MODEL_PACKAGE_GROUP

'dogBreeds'

In [47]:
event_pattern = f"""
{{
  "source": ["aws.sagemaker"],
  "detail-type": ["SageMaker Model Package State Change"],
  "detail": {{
    "ModelPackageGroupName": ["{MODEL_PACKAGE_GROUP}"],
    "ModelApprovalStatus": ["Approved"]
  }}
}}
"""

In [50]:
event_pattern

'\n{\n  "source": ["aws.sagemaker"],\n  "detail-type": ["SageMaker Model Package State Change"],\n  "detail": {\n    "ModelPackageGroupName": ["dogBreeds"],\n    "ModelApprovalStatus": ["Approved"]\n  }\n}\n'

In [48]:
events_client = boto3.client("events")
rule_response = events_client.put_rule(
    Name="PipelineModelApprovedRule",
    EventPattern=event_pattern,
    State="ENABLED",
    RoleArn=role,
)

ClientError: An error occurred (AccessDeniedException) when calling the PutRule operation: User: arn:aws:sts::681340771742:assumed-role/AmazonSageMaker-ExecutionRole-20230916T122655/SageMaker is not authorized to perform: events:PutRule on resource: arn:aws:events:us-east-1:681340771742:rule/PipelineModelApprovedRule because no identity-based policy allows the events:PutRule action

# Deploy

In [4]:
bucket = "pochingto-testing"

In [8]:
from sagemaker.predictor import Predictor

ENDPOINT = "dogBreeds-endpoint"
DATA_CAPTURE_DESTINATION = f"{bucket}/monitoring/data-capture"
MODEL_PACKAGE_GROUP = "dogBreeds"

In [13]:
import boto3
import sagemaker

sagemaker_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [14]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = (
    response["ModelPackageSummaryList"][0]
    if response["ModelPackageSummaryList"]
    else None
)
package

{'ModelPackageGroupName': 'dogBreeds',
 'ModelPackageVersion': 5,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:681340771742:model-package/dogBreeds/5',
 'CreationTime': datetime.datetime(2023, 11, 3, 2, 56, 10, 102000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'Approved'}

In [15]:
from sagemaker import ModelPackage

model_package = ModelPackage(
    model_package_arn=package["ModelPackageArn"],
    sagemaker_session=sagemaker_session,
    role=role,
)

In [16]:
# model_package.deploy(
#     endpoint_name=ENDPOINT, 
#     initial_instance_count=1, 
#     instance_type=config["instance_type"]
# )

In [17]:
import time
import boto3

sagemaker_client = boto3.client("sagemaker")

endpoint_name = "dogBreeds-endpoint"
data_capture_destination = f"s3://{bucket}/monitoring/data-capture"

timestamp = time.strftime("%m%d%H%M%S", time.localtime())
model_name = f"{endpoint_name}-model-{timestamp}"
endpoint_config_name = f"{endpoint_name}-config-{timestamp}"
model_package_arn=package["ModelPackageArn"]

sagemaker_client.create_model(
    ModelName=model_name, 
    ExecutionRoleArn=role, 
    Containers=[{
        "ModelPackageName": model_package_arn
    }] 
)
sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        "ModelName": model_name,
        "InstanceType": "ml.m5.xlarge",
        "InitialVariantWeight": 1,
        "InitialInstanceCount": 1,
        "VariantName": "AllTraffic",
    }],

    DataCaptureConfig={
        "EnableCapture": True,
        "InitialSamplingPercentage": 100,
        "DestinationS3Uri": data_capture_destination,
        "CaptureOptions": [
            {
                "CaptureMode": "Input"
            },
            {
                "CaptureMode": "Output"
            },
        ],
        "CaptureContentTypeHeader": {
            "JsonContentTypes": [
                "application/json",
                "application/x-image"
            ]
        }
    },
)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:681340771742:endpoint-config/dogbreeds-endpoint-config-1103183233',
 'ResponseMetadata': {'RequestId': 'e58dde11-b8f3-4020-a9f3-b5fed768e9ff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e58dde11-b8f3-4020-a9f3-b5fed768e9ff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '117',
   'date': 'Fri, 03 Nov 2023 18:32:34 GMT'},
  'RetryAttempts': 0}}

In [18]:
response = sagemaker_client.list_endpoints(NameContains=endpoint_name, MaxResults=1)

if len(response["Endpoints"]) == 0:
    sagemaker_client.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name,
    )
else:
    sagemaker_client.update_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name,
    )

## Evaluate

In [22]:
from PIL import Image
import io

def load_and_preprocess_image(image_path):
    # Load the image
    image = Image.open(image_path)
    # image = image.resize((224, 224))

    # Convert the image to bytes
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='JPEG')  # Adjust format if needed
    img_byte_arr = img_byte_arr.getvalue()

    return img_byte_arr

image_path = './test_images/australian-shepherd.jpg'
image_data = load_and_preprocess_image(image_path)

In [23]:
# image_data

In [24]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

content_type = "application/x-image"

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=image_data
)

# Parse the response
response_body = response['Body'].read()
predictions = json.loads(response_body)
print(predictions)

Australian_shepherd
